In [1]:
import qiskit as qk
import trueq as tq
from qiskit import IBMQ
import trueq.compilation as tqc
from qiskit import Aer

In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider()

In [3]:
backend = provider.get_backend('ibm_oslo')

In [4]:
c = qk.QuantumCircuit(4)
# c.h(0)
# c.cx(0, 1)
# c.h(3)
c.x(3)
c.measure_all()
c = qk.transpile(c, backend)

In [5]:
c = qk.circuit.library.QFT(num_qubits=4)
c.measure_all()
c = qk.transpile(c, backend)

In [6]:
c.draw()

global phase: 4.8106
                                                                             »
ancilla_0 -> 0 ──────────────────────────────────────────────────────────────»
                                                                             »
ancilla_1 -> 1 ──────────────────────────────────────────────────────────────»
                                                                             »
ancilla_2 -> 2 ──────────────────────────────────────────────────────────────»
               ┌─────────┐┌────┐┌──────────┐                                 »
      q_3 -> 3 ┤ Rz(π/2) ├┤ √X ├┤ Rz(3π/4) ├──■────────────────■─────────────»
               └─────────┘└────┘└──────────┘  │                │             »
      q_0 -> 4 ───────────────────────────────┼────────────────┼─────────────»
                                            ┌─┴─┐┌──────────┐┌─┴─┐┌─────────┐»
      q_2 -> 5 ─────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├»
                                            └───┘└──────────┘└───┘└─────────┘»
      q_1 -> 6 ──────────────────────────────────────────────────────────────»
                                                                             »
       meas: 4/══════════════════════════════════════════════════════════════»
                                                                             »
«                                                                           »
«ancilla_0 -> 0 ────────────────────────────────────────────────────────────»
«                                                                           »
«ancilla_1 -> 1 ────────────────────────────────────────────────────────────»
«                                                                           »
«ancilla_2 -> 2 ────────────────────────────────────────────────────────────»
«                    ┌───┐     ┌─────────┐┌────┐┌──────────┐                »
«      q_3 -> 3 ──■──┤ X ├──■──┤ Rz(π/2) ├┤ √X ├┤ Rz(3π/4) ├────────────────»
«                 │  └─┬─┘  │  └─────────┘└────┘└──────────┘                »
«      q_0 -> 4 ──┼────┼────┼───────────────────────────────────────────────»
«               ┌─┴─┐  │  ┌─┴─┐┌─────────┐                                  »
«      q_2 -> 5 ┤ X ├──■──┤ X ├┤ Rz(π/8) ├──■─────────────────■─────────────»
«               └───┘     └───┘└─────────┘┌─┴─┐ ┌──────────┐┌─┴─┐┌─────────┐»
«      q_1 -> 6 ──────────────────────────┤ X ├─┤ Rz(-π/8) ├┤ X ├┤ Rz(π/8) ├»
«                                         └───┘ └──────────┘└───┘└─────────┘»
«       meas: 4/════════════════════════════════════════════════════════════»
«                                                                           »
«                                                                           »
«ancilla_0 -> 0 ────────────────────────────────────────────────────────────»
«                                                                           »
«ancilla_1 -> 1 ────────────────────────────────────────────────────────────»
«                                                                           »
«ancilla_2 -> 2 ────────────────────────────────────────────────────────────»
«                                                           ┌─────────┐     »
«      q_3 -> 3 ────────────────────■────────────────────■──┤ Rz(π/8) ├─────»
«                                   │                    │  └─────────┘┌───┐»
«      q_0 -> 4 ────────────────────┼────────────────────┼─────────────┤ X ├»
«                    ┌───┐        ┌─┴─┐    ┌──────────┐┌─┴─┐┌─────────┐└─┬─┘»
«      q_2 -> 5 ──■──┤ X ├──■─────┤ X ├────┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├──■──»
«               ┌─┴─┐└─┬─┘┌─┴─┐┌──┴───┴───┐└──────────┘└───┘└─────────┘     »
«      q_1 -> 6 ┤ X ├──■──┤ X ├┤ Rz(π/16) ├─────────────────────────────────»
«               └───┘     └───┘└──────────┘                                 »
«       meas: 4/════════════════════════════════════════════════════════════»
«                                                             

In [40]:
def flip_bits(counts_dict):
    new_dict = dict()
    for key, val in counts_dict.items():
        new_dict[key[::-1]] = val
    return new_dict

def generate_mitigated_res(n_qubits, tq_collection, n_shots):
    res_dict = dict()
    for k in range(2 ** n_qubits):
        bs = f'{k:0{n_qubits}b}'
        prob = nox_collection_trueq.fit(observables=[bs]).estimates[0]._values[0]
        res_dict[bs] = int(n_shots * prob) if prob > 0 else 0
    return res_dict

tq.interface.qiskit.set_from_backend(backend)
circuit_tq, metadata = tq.interface.qiskit.to_trueq_circ(c)

circuit_tq = tqc.UnmarkCycles().apply(circuit_tq)

nox_collection_trueq = tq.make_nox(circuit_tq)
nox_collection_qiskit = [tq.interface.qiskit.from_trueq_circ(cir, metadata=metadata) for cir in nox_collection_trueq]

In [41]:
sim = Aer.get_backend("aer_simulator")
job = sim.run(nox_collection_qiskit, num_shots=1000)
res = job.result()
counts = res.get_counts()
for idx, circuit in enumerate(nox_collection_trueq):
    circuit.results = flip_bits(counts[idx])

In [42]:
mit_res = generate_mitigated_res(n_qubits=4, tq_collection = nox_collection_trueq, n_shots=1000)

In [21]:
res.results = mit_res

In [25]:
res

Result(backend_name='aer_simulator', backend_version='0.11.0', qobj_id='f5f4952f-ba81-4523-b807-73b3e346771d', job_id='c786e6a6-61e5-4dbc-9aa4-df673633d15c', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x9': 56, '0x5': 65, '0xa': 54, '0xf': 63, '0xd': 65, '0x8': 67, '0xe': 68, '0x3': 67, '0x1': 64, '0x6': 62, '0xc': 72, '0x4': 61, '0xb': 61, '0x7': 77, '0x0': 68, '0x2': 54}), header=QobjExperimentHeader(clbit_labels=[['meas', 0], ['meas', 1], ['meas', 2], ['meas', 3]], creg_sizes=[['meas', 4]], global_phase=0.0, memory_slots=4, metadata={}, n_qubits=7, name='circuit-141', qreg_sizes=[['q', 7]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4], ['q', 5], ['q', 6]]), status=DONE, seed_simulator=2380545406, metadata={'parallel_state_update': 16, 'parallel_shots': 1, 'sample_measure_time': 0.000356783, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': True, 'device': 'CPU', 'active_

In [43]:
res_copy = copy.copy(res.results[0])

res_copy.data.counts = mit_res

In [44]:
res_copy

ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0000': 67, '0001': 54, '0010': 67, '0011': 50, '0100': 64, '0101': 27, '0110': 45, '0111': 91, '1000': 47, '1001': 6, '1010': 98, '1011': 97, '1100': 104, '1101': 62, '1110': 41, '1111': 72}), header=QobjExperimentHeader(clbit_labels=[['meas', 0], ['meas', 1], ['meas', 2], ['meas', 3]], creg_sizes=[['meas', 4]], global_phase=0.0, memory_slots=4, metadata={}, n_qubits=7, name='circuit-1163', qreg_sizes=[['q', 7]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4], ['q', 5], ['q', 6]]), status=DONE, seed_simulator=2778338246, metadata={'parallel_state_update': 16, 'parallel_shots': 1, 'sample_measure_time': 0.000350766, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': True, 'device': 'CPU', 'active_input_qubits': [3, 4, 5, 6], 'measure_sampling': True, 'num_clbits': 4, 'input_qubit_map': [[6, 3], [5, 2], [4, 1], [3, 0]], 'num_qubits': 4, 'method': 'statevector

In [45]:
ret = copy.copy(res)

In [46]:
ret.results = [res_copy]

In [47]:
ret.get_counts()

{'0000': 67,
 '0001': 54,
 '0010': 67,
 '0011': 50,
 '0100': 64,
 '0101': 27,
 '0110': 45,
 '0111': 91,
 '1000': 47,
 '1001': 6,
 '1010': 98,
 '1011': 97,
 '1100': 104,
 '1101': 62,
 '1110': 41,
 '1111': 72}

In [48]:
mit_res

{'0000': 67,
 '0001': 54,
 '0010': 67,
 '0011': 50,
 '0100': 64,
 '0101': 27,
 '0110': 45,
 '0111': 91,
 '1000': 47,
 '1001': 6,
 '1010': 98,
 '1011': 97,
 '1100': 104,
 '1101': 62,
 '1110': 41,
 '1111': 72}

In [55]:
res.results[0].shots

1024

In [56]:
ret

Result(backend_name='aer_simulator', backend_version='0.11.0', qobj_id='e03bcb9f-21d6-4ded-bf9a-4c69a565cb80', job_id='c2ae491f-cb5c-493e-a66c-a149e9061904', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0000': 67, '0001': 54, '0010': 67, '0011': 50, '0100': 64, '0101': 27, '0110': 45, '0111': 91, '1000': 47, '1001': 6, '1010': 98, '1011': 97, '1100': 104, '1101': 62, '1110': 41, '1111': 72}), header=QobjExperimentHeader(clbit_labels=[['meas', 0], ['meas', 1], ['meas', 2], ['meas', 3]], creg_sizes=[['meas', 4]], global_phase=0.0, memory_slots=4, metadata={}, n_qubits=7, name='circuit-1163', qreg_sizes=[['q', 7]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4], ['q', 5], ['q', 6]]), status=DONE, seed_simulator=2778338246, metadata={'parallel_state_update': 16, 'parallel_shots': 1, 'sample_measure_time': 0.000350766, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': True, 'device'